1 subject. Feature extraction. Classification for each frequency band.

In [643]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
'''
files = os.listdir('d\\')

for file in files:
    with open(os.path.join('d\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
        
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        allData = allData.merge(aux, left_index=True, right_index=True)
        
        list_of_dfs.append(allData)
data = pd.concat(list_of_dfs, ignore_index=True)

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

data
'''

'\nfiles = os.listdir(\'d\\\')\n\nfor file in files:\n    with open(os.path.join(\'d\\\',file), \'rb\') as pickle_file:\n        dictRaw = pd.read_pickle(pickle_file) \n        labels = dictRaw.get(\'labels\')\n        data = dictRaw.get(\'data\')\n        \n        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])\n\n        a,b,c = data.shape\n        E = data.reshape(40, 322560)\n        allData = pd.DataFrame(E).transpose()\n        rows = list()\n        for _,row in dfLabels.iterrows():\n            rows += [row]*8064\n        aux = pd.DataFrame(rows).reset_index(drop=True)\n        allData = allData.merge(aux, left_index=True, right_index=True)\n        \n        list_of_dfs.append(allData)\ndata = pd.concat(list_of_dfs, ignore_index=True)\n\ndata.columns = [\'FP1\',\'AF3\',\'F7\',\'F3\',\'FC1\',\'FC5\',\'T7\',\'C3\',\'CP1\',\'CP5\',\'P7\',\'P3\',\'Pz\',\'PO3\',\'O1\',\'Oz\',\'O2\',\'PO4\',\'P4\',\'P8\',\'CP6\',\'CP2\',\'C4\',\'T8\',\'FC

In [644]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#def select_file(file):
    #Lectura mediante pandas
f = "data_preprocessed_python\\s32.dat"
subject_id = 32

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)
#df_videos

In [645]:
    # Frecuencia de muestreo
    fs = 128
    # Window
    window = "hann"
    # Length of each segment
    # nperseg = 256 por defecto
    # noverlap
    # Por defecto a None, if None : noverlap = nperseg / 2
    # [...]

    # Definicion de bandas
    eeg_bands = {'Delta': (1, 4),
                'Theta': (4, 8),
                'Alpha': (8, 14),
                'Beta': (14, 30),
                'Gamma': (30, 50)}

In [646]:
# FOURIER
import scipy
from scipy import signal

# SE COGE EL PRIMER VÍDEOS, Df_VIDEOS[0]
def do_stft(video, channel):
    # Array 
    x = df_videos[video][channel]

    # Array of the sample frequency, Array of the segment times, STFT of x
    f, t , Zxx = scipy.signal.stft(x, fs, window)

    #print(np.abs(Zxx))
    '''
    plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    '''
    return Zxx


In [647]:
#SEPARACION EN LAS DISTINTAS BANDAS DE FRECUENCIA

def make_bands(Zxx):
    #Obtener valores reales de STFT, solo positivos.
    values = np.absolute(Zxx)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    #print(fft_freq)

    # Obtención de cada una de las bandas de frecuencia
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = values[freq_ix]

    #print(eeg_band_fft)
    '''
    eeg_band_fft_means = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft_means[band] = np.mean(values[freq_ix])

    #print(eeg_band_fft_means)

    #Graficación de las medias de los datos
    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft_means[band] for band in eeg_bands]
    ax = df.plot.bar(x='band', y='val', legend=False)
    ax.set_xlabel("EEG band")
    ax.set_ylabel("Mean band Amplitude")
    '''
    #print(df)

    #print("\n\n")

    '''
    print("Shape de Zxx: ", Zxx.shape)
    print("Shape de Delta: ", eeg_band_fft['Delta'].shape)
    print("Shape de Theta: ", eeg_band_fft['Theta'].shape)
    print("Shape de Alpha: ", eeg_band_fft['Alpha'].shape)
    print("Shape de Beta: ", eeg_band_fft['Beta'].shape)
    print("Shape de Gamma: ", eeg_band_fft['Gamma'].shape)
    '''
    return eeg_band_fft


In [648]:
# Ahora hay que hacer las medias sobre cada banda de frecuencia. La media entre cada una de las posiciones de los bloques. Hacer la media entre todas las posiciones 0, 1, 2... 63 de cada uno de los bloques
import numpy as np

def channel_freq(eeg_band_fft):
        
        freq = np.zeros((5, 64))        # 5 bandas, 64 posiciones de los datos

        # Se va a recorrer {Delta, Theta, Alpha, Beta, Gamma}
        for band in eeg_bands:

                # Para cada una de las 64 posiciones del array
                for j in range (0, 64):
                        
                        val = []
                        # Se va a recorrer cada uno de los arrays que hay en cada una de las bandas
                        for i in range (0, len(eeg_band_fft[band])):
                                val.append(eeg_band_fft[band][i][j])
                                # Ejemplo : eeg_band_fft['Delta'][0][0*0 + 0]
                        if (band == "Delta"):
                                freq[0][j] = np.mean(val)
                        elif (band == "Theta"):
                                freq[1][j] = np.mean(val)
                        elif (band == "Alpha"):
                                freq[2][j] = np.mean(val)
                        elif (band == "Beta"):
                                freq[3][j] = np.mean(val)
                        elif (band == "Gamma"):
                                freq[4][j] = np.mean(val)
        return(freq)     # En freq tenemos las frecuencias medias de cada una de las bandas

# Recordamos : Estamos tratando los datos relativos al video 0, un canal concreto de un sujeto concreto

In [649]:
#df_file_videos = select_file('d\\s01.dat')
# Obtener la división por bandas de frecuencia de cada uno de los canales del video 0
for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        print(video)
        print("\n")
        print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

0


FP1
0


AF3
0


F7
0


F3
0


FC1
0


FC5
0


T7
0


C3
0


CP1
0


CP5
0


P7
0


P3
0


Pz
0


PO3
0


O1
0


Oz
0


O2
0


PO4
0


P4
0


P8
0


CP6
0


CP2
0


C4
0


T8
0


FC6
0


FC2
0


F4
0


F8
0


AF4
0


Fp2
0


Fz
0


Cz
1


FP1
1


AF3
1


F7
1


F3
1


FC1
1


FC5
1


T7
1


C3
1


CP1
1


CP5
1


P7
1


P3
1


Pz
1


PO3
1


O1
1


Oz
1


O2
1


PO4
1


P4
1


P8
1


CP6
1


CP2
1


C4
1


T8
1


FC6
1


FC2
1


F4
1


F8
1


AF4
1


Fp2
1


Fz
1


Cz
2


FP1
2


AF3
2


F7
2


F3
2


FC1
2


FC5
2


T7
2


C3
2


CP1
2


CP5
2


P7
2


P3
2


Pz
2


PO3
2


O1
2


Oz
2


O2
2


PO4
2


P4
2


P8
2


CP6
2


CP2
2


C4
2


T8
2


FC6
2


FC2
2


F4
2


F8
2


AF4
2


Fp2
2


Fz
2


Cz
3


FP1
3


AF3
3


F7
3


F3
3


FC1
3


FC5
3


T7
3


C3
3


CP1
3


CP5
3


P7
3


P3
3


Pz
3


PO3
3


O1
3


Oz
3


O2
3


PO4
3


P4
3


P8
3


CP6
3


CP2
3


C4
3


T8
3


FC6
3


FC2
3


F4
3


F8
3


AF4
3


Fp2
3


Fz
3


Cz
4


FP1
4


AF3
4


F7
4


F3
4


FC1
4


FC5
4


In [650]:
# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

aux_delta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
1,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
2,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
3,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
4,0.025366,0.078844,0.039507,0.094259,0.044156,0.037892,0.034969,0.148271,0.164957,0.033372,...,0.372899,0.035648,0.065213,0.052704,0.213639,0.099711,0.048500,0.074329,0.140679,0.029246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.022481,0.172611,0.057046,0.109849,0.242265,0.070911,0.196101,0.334543,0.355690,0.188437,...,1.544856,0.022754,0.107385,0.062702,0.663288,0.398949,0.165376,0.216283,0.609451,0.048894
2556,0.022481,0.172611,0.057046,0.109849,0.242265,0.070911,0.196101,0.334543,0.355690,0.188437,...,1.544856,0.022754,0.107385,0.062702,0.663288,0.398949,0.165376,0.216283,0.609451,0.048894
2557,0.022481,0.172611,0.057046,0.109849,0.242265,0.070911,0.196101,0.334543,0.355690,0.188437,...,1.544856,0.022754,0.107385,0.062702,0.663288,0.398949,0.165376,0.216283,0.609451,0.048894
2558,0.022481,0.172611,0.057046,0.109849,0.242265,0.070911,0.196101,0.334543,0.355690,0.188437,...,1.544856,0.022754,0.107385,0.062702,0.663288,0.398949,0.165376,0.216283,0.609451,0.048894


In [651]:
# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

aux_theta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
1,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
2,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
3,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
4,0.018908,0.119647,0.045046,0.120408,0.052348,0.033803,0.062029,0.194587,0.230034,0.050673,...,0.575209,0.041922,0.079025,0.087149,0.303126,0.145937,0.071534,0.109496,0.221562,0.047842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.024220,0.166680,0.067325,0.147044,0.244436,0.073475,0.191783,0.327033,0.346295,0.224839,...,1.607303,0.034971,0.120223,0.043455,0.680246,0.414566,0.172088,0.201217,0.634894,0.061363
2556,0.024220,0.166680,0.067325,0.147044,0.244436,0.073475,0.191783,0.327033,0.346295,0.224839,...,1.607303,0.034971,0.120223,0.043455,0.680246,0.414566,0.172088,0.201217,0.634894,0.061363
2557,0.024220,0.166680,0.067325,0.147044,0.244436,0.073475,0.191783,0.327033,0.346295,0.224839,...,1.607303,0.034971,0.120223,0.043455,0.680246,0.414566,0.172088,0.201217,0.634894,0.061363
2558,0.024220,0.166680,0.067325,0.147044,0.244436,0.073475,0.191783,0.327033,0.346295,0.224839,...,1.607303,0.034971,0.120223,0.043455,0.680246,0.414566,0.172088,0.201217,0.634894,0.061363


In [652]:
#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

aux_alpha

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
1,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
2,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
3,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
4,0.018843,0.164569,0.063804,0.156485,0.093022,0.038792,0.091270,0.236123,0.288100,0.073060,...,0.779159,0.043292,0.105756,0.133018,0.398716,0.203477,0.103539,0.140532,0.303441,0.065347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.037833,0.188339,0.094489,0.242650,0.306649,0.087516,0.219157,0.374257,0.418595,0.349438,...,2.126998,0.055798,0.156946,0.060423,0.849186,0.512698,0.201694,0.223133,0.830502,0.104939
2556,0.037833,0.188339,0.094489,0.242650,0.306649,0.087516,0.219157,0.374257,0.418595,0.349438,...,2.126998,0.055798,0.156946,0.060423,0.849186,0.512698,0.201694,0.223133,0.830502,0.104939
2557,0.037833,0.188339,0.094489,0.242650,0.306649,0.087516,0.219157,0.374257,0.418595,0.349438,...,2.126998,0.055798,0.156946,0.060423,0.849186,0.512698,0.201694,0.223133,0.830502,0.104939
2558,0.037833,0.188339,0.094489,0.242650,0.306649,0.087516,0.219157,0.374257,0.418595,0.349438,...,2.126998,0.055798,0.156946,0.060423,0.849186,0.512698,0.201694,0.223133,0.830502,0.104939


In [653]:
# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

aux_beta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
1,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
2,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
3,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
4,0.070166,1.060305,0.308343,0.835856,0.636678,0.270968,0.687164,1.028565,1.138698,0.257403,...,3.746501,0.065353,0.510457,0.789394,1.730486,1.008250,0.711025,0.596827,1.678863,0.424949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.244119,0.542509,0.365433,1.300983,0.742717,0.264139,0.398248,1.000291,0.857970,1.323068,...,6.083535,0.337323,0.717518,0.228581,2.159327,1.279826,0.506622,0.282298,2.389439,0.408206
2556,0.244119,0.542509,0.365433,1.300983,0.742717,0.264139,0.398248,1.000291,0.857970,1.323068,...,6.083535,0.337323,0.717518,0.228581,2.159327,1.279826,0.506622,0.282298,2.389439,0.408206
2557,0.244119,0.542509,0.365433,1.300983,0.742717,0.264139,0.398248,1.000291,0.857970,1.323068,...,6.083535,0.337323,0.717518,0.228581,2.159327,1.279826,0.506622,0.282298,2.389439,0.408206
2558,0.244119,0.542509,0.365433,1.300983,0.742717,0.264139,0.398248,1.000291,0.857970,1.323068,...,6.083535,0.337323,0.717518,0.228581,2.159327,1.279826,0.506622,0.282298,2.389439,0.408206


In [654]:
# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

aux_gamma

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
1,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
2,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
3,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
4,0.296230,0.425451,0.276974,0.417305,0.571668,0.284168,0.424563,0.884798,0.792988,0.438355,...,2.431686,0.295712,0.348758,0.779843,1.044175,0.580807,0.359518,0.834561,0.960447,0.406656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.370961,0.449814,1.146706,3.372740,2.326301,0.212740,0.593154,1.589803,2.557764,3.804570,...,17.305105,0.771617,1.971791,0.788704,5.179255,3.076131,0.743108,0.665851,6.491435,1.554092
2556,0.370961,0.449814,1.146706,3.372740,2.326301,0.212740,0.593154,1.589803,2.557764,3.804570,...,17.305105,0.771617,1.971791,0.788704,5.179255,3.076131,0.743108,0.665851,6.491435,1.554092
2557,0.370961,0.449814,1.146706,3.372740,2.326301,0.212740,0.593154,1.589803,2.557764,3.804570,...,17.305105,0.771617,1.971791,0.788704,5.179255,3.076131,0.743108,0.665851,6.491435,1.554092
2558,0.370961,0.449814,1.146706,3.372740,2.326301,0.212740,0.593154,1.589803,2.557764,3.804570,...,17.305105,0.771617,1.971791,0.788704,5.179255,3.076131,0.743108,0.665851,6.491435,1.554092


In [655]:
df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

aux_labels

,valence,arousal,dominance,liking
0,8.13,4.83,9.0,4.87
1,8.13,4.83,9.0,4.87
2,8.13,4.83,9.0,4.87
3,8.13,4.83,9.0,4.87
4,8.13,4.83,9.0,4.87
...,...,...,...,...
2555,7.15,4.03,9.0,1.88
2556,7.15,4.03,9.0,1.88
2557,7.15,4.03,9.0,1.88
2558,7.15,4.03,9.0,1.88


In [656]:
data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

In [657]:
data = data_delta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score


accuracy_score(yTest, predict)*100
    
from sklearn import svm
from sklearn.svm import LinearSVC

svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(xTrain, np.ravel(yTrain))
knn_predict = knn.predict(xTest)

rf_acc_delta = accuracy_score(yTest, predict)
svm_acc_delta = accuracy_score(yTest, svm_predict)
knn_acc_delta = accuracy_score(yTest, knn_predict)

<ipython-input-657-23530eff5e45>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


In [658]:
data = data_theta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score

from sklearn import svm
from sklearn.svm import LinearSVC

svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(xTrain, np.ravel(yTrain))
knn_predict = knn.predict(xTest)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

<ipython-input-658-615b103c0788>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


In [659]:
data = data_alpha.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import LinearSVC

svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(xTrain, np.ravel(yTrain))
knn_predict = knn.predict(xTest)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

<ipython-input-659-84cbb6679702>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


In [660]:
data = data_beta.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import LinearSVC

svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(xTrain, np.ravel(yTrain))
knn_predict = knn.predict(xTest)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

<ipython-input-660-d1a8bafe4965>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
C:\Python39\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [661]:
data = data_gamma.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','valence']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import LinearSVC

svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(xTrain, np.ravel(yTrain))
knn_predict = knn.predict(xTest)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

<ipython-input-661-949ffa55bc62>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
C:\Python39\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [662]:
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy']

data_CSV = [subject_id, 2, "STFT", "Delta", "-", "Random Forest", rf_acc_delta], [subject_id, 2, "STFT", "Delta", "-", "SVM", svm_acc_delta], [subject_id, 2, "STFT", "Delta", "-", "kNN", knn_acc_delta], [subject_id, 2, "STFT", "Theta", "-", "Random Forest", rf_acc_theta], [subject_id, 2, "STFT", "Theta", "-", "SVM", svm_acc_theta], [subject_id, 2, "STFT", "Theta", "-", "kNN", knn_acc_theta], [subject_id, 2, "STFT", "Alpha", "-", "Random Forest", rf_acc_alpha], [subject_id, 2, "STFT", "Alpha", "-", "SVM", svm_acc_alpha], [subject_id, 2, "STFT", "Alpha", "-", "kNN", knn_acc_alpha], [subject_id, 2, "STFT", "Beta", "-", "Random Forest", rf_acc_beta], [subject_id, 2, "STFT", "Beta", "-", "SVM", svm_acc_beta], [subject_id, 2, "STFT", "Beta", "-", "kNN", knn_acc_beta], [subject_id, 2, "STFT", "Gamma", "-", "Random Forest", rf_acc_gamma], [subject_id, 2, "STFT", "Gamma", "-", "SVM", svm_acc_gamma], [subject_id, 2, "STFT", "Gamma", "-", "kNN", knn_acc_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment2.csv", mode="a", header=False, index=False)